<a href="https://colab.research.google.com/github/Anjasfedo/Code-as-a-Cryptography/blob/main/koblitz_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# y^2 = x^3 + 7 mod 17
def curve_equation(x, a=-1, b=188, p=751):
    """Elliptic curve equation y^2 = x^3 + ax + b mod p."""
    print((x**3 + a*x + b) % p)
    return (x**3 + a*x + b) % p

def is_curve_point(x, y, a=-1, b=188, p=751):
    """Check if the point (x, y) satisfies the elliptic curve equation mod p."""
    print(f"{curve_equation(x, a, b, p)} == {(y**2) % p}")
    return curve_equation(x, a, b, p) == (y**2) % p

## Koblitz Method

In [15]:
is_curve_point(224, 248)

673
673 == 673
673


True

In [12]:
m = 11
k = 20

In [13]:
x1 = m * k + 1
x1

221

In [17]:
is_curve_point(x1, curve_equation(x1))

456
456
456 == 660
456


False

In [18]:
x2 = m * k + 2
x2

222

In [19]:
is_curve_point(x2, curve_equation(x2))

446
446
446 == 652
446


False

In [20]:
x3 = m * k + 3
x3

223

In [21]:
is_curve_point(x3, curve_equation(x3))

266
266
266 == 162
266


False

In [22]:
x4 = m * k + 4
x4

224

In [24]:
is_curve_point(x4, x4)

673
673 == 610
673


False

In [34]:
import sympy as sp

# Parameters
p = 751  # prime modulus
k = 20    # chosen small integer
a = -1    # coefficient of x in the elliptic curve
b = 188    # constant in the elliptic curve

# Elliptic curve equation: y^2 = x^3 + ax + b mod p
def koblitz_encode(m):
    num = 1
    point = ()

    while not point:
      x = m * k + num
      rhs = (x**3 + a * x + b) % p  # right-hand side of the elliptic curve equation

      # Check if rhs is a quadratic residue modulo p
      if sp.is_quad_residue(rhs, p):
          y = sp.sqrt_mod(rhs, p)
          point = (x, y)  # return the point (x, y)
      else:
          point = ()  # no valid point found, try another m

      num += 1

    return point

# Decoding: m = (x - 1) / k
def koblitz_decode(x):
    return (x - 1) // k

# Example: encoding message m
message = 11
encoded_point = koblitz_encode(message)

# Output the result
decoded_message = koblitz_decode(encoded_point[0])
print(f"Encoded point: {encoded_point}")
print(f"Decoded message: {decoded_message}")

Encoded point: (224, 248)
Decoded message: 11


In [35]:
import sympy as sp

# Parameters
p = 751  # prime modulus
k = 20   # chosen small integer
a = -1   # coefficient of x in the elliptic curve
b = 188  # constant in the elliptic curve

# Elliptic curve equation: y^2 = x^3 + ax + b mod p
def koblitz_encode(m, max_attempts=1000):
    num = 1  # Start with x = m * k + 1
    attempts = 0

    while attempts < max_attempts:
        x = m * k + num
        rhs = (x**3 + a * x + b) % p  # right-hand side of the elliptic curve equation

        # Check if rhs is a quadratic residue modulo p
        if sp.is_quad_residue(rhs, p):
            y = sp.sqrt_mod(rhs, p)
            return (x, y)  # Return the point (x, y) as a tuple

        num += 1  # Increment to check next x value
        attempts += 1

    # If no valid point is found after max_attempts
    raise ValueError(f"No valid point found after {max_attempts} attempts for message {m}.")

# Decoding: m = (x - num) / k
def koblitz_decode(x, num=1):
    return (x - num) // k

# Example: encoding message m
message = 11
encoded_point = koblitz_encode(message)

# Decoding message
decoded_message = koblitz_decode(encoded_point[0])

# Output the result
print(f"Encoded point: {encoded_point}")
print(f"Decoded message: {decoded_message}")


Encoded point: (224, 248)
Decoded message: 11


In [36]:
# Extended Euclidean Algorithm to find the inverse mod p
def mod_inverse(a, p):
    return pow(a, p - 2, p)

# Function to check if rhs is a quadratic residue mod p (Euler's criterion)
def is_quadratic_residue(rhs, p):
    return pow(rhs, (p - 1) // 2, p) == 1

# Function to compute modular square root using Tonelli-Shanks algorithm
def mod_sqrt(rhs, p):
    # Special case for p == 2
    if p == 2:
        return rhs % 2

    # Check if rhs is a quadratic residue mod p
    if not is_quadratic_residue(rhs, p):
        return None  # No square root exists

    # For p ≡ 3 (mod 4), we can compute the square root directly
    if p % 4 == 3:
        return pow(rhs, (p + 1) // 4, p)

    # Tonelli-Shanks algorithm for general prime p
    # Find q and s such that p - 1 = q * 2^s with q odd
    s = 0
    q = p - 1
    while q % 2 == 0:
        q //= 2
        s += 1

    # Find a non-quadratic residue z
    z = 2
    while is_quadratic_residue(z, p):
        z += 1

    m = s
    c = pow(z, q, p)
    t = pow(rhs, q, p)
    r = pow(rhs, (q + 1) // 2, p)

    while t != 0 and t != 1:
        t2i = t
        i = 0
        for i in range(1, m):
            t2i = pow(t2i, 2, p)
            if t2i == 1:
                break

        b = pow(c, 1 << (m - i - 1), p)
        m = i
        c = pow(b, 2, p)
        t = (t * c) % p
        r = (r * b) % p

    return r

# Parameters
p = 751  # prime modulus
k = 20   # chosen small integer
a = -1   # coefficient of x in the elliptic curve
b = 188  # constant in the elliptic curve

# Elliptic curve equation: y^2 = x^3 + ax + b mod p
def koblitz_encode(m, max_attempts=1000):
    num = 1  # Start with x = m * k + 1
    attempts = 0

    while attempts < max_attempts:
        x = m * k + num
        rhs = (x**3 + a * x + b) % p  # right-hand side of the elliptic curve equation

        # Check if rhs is a quadratic residue modulo p
        if is_quadratic_residue(rhs, p):
            y = mod_sqrt(rhs, p)
            if y is not None:
                return (x, y)  # Return the point (x, y) as a tuple

        num += 1  # Increment to check next x value
        attempts += 1

    # If no valid point is found after max_attempts
    raise ValueError(f"No valid point found after {max_attempts} attempts for message {m}.")

# Decoding: m = (x - num) / k
def koblitz_decode(x, num=1):
    return (x - num) // k

# Example: encoding message m
message = 11
encoded_point = koblitz_encode(message)

# Decoding message
decoded_message = koblitz_decode(encoded_point[0])

# Output the result
print(f"Encoded point: {encoded_point}")
print(f"Decoded message: {decoded_message}")


Encoded point: (224, 503)
Decoded message: 11


In [37]:
# Extended Euclidean Algorithm to find the inverse mod p
def mod_inverse(a, p):
    return a ** (p - 2) % p

# Function to check if rhs is a quadratic residue mod p (Euler's criterion)
def is_quadratic_residue(rhs, p):
    return rhs ** ((p - 1) // 2) % p == 1

# Function to compute modular square root using Tonelli-Shanks algorithm
def mod_sqrt(rhs, p):
    # Special case for p == 2
    if p == 2:
        return rhs % 2

    # Check if rhs is a quadratic residue mod p
    if not is_quadratic_residue(rhs, p):
        return None  # No square root exists

    # For p ≡ 3 (mod 4), we can compute the square root directly
    if p % 4 == 3:
        return rhs ** ((p + 1) // 4) % p

    # Tonelli-Shanks algorithm for general prime p
    # Find q and s such that p - 1 = q * 2^s with q odd
    s = 0
    q = p - 1
    while q % 2 == 0:
        q //= 2
        s += 1

    # Find a non-quadratic residue z
    z = 2
    while is_quadratic_residue(z, p):
        z += 1

    m = s
    c = z ** q % p
    t = rhs ** q % p
    r = rhs ** ((q + 1) // 2) % p

    while t != 0 and t != 1:
        t2i = t
        i = 0
        for i in range(1, m):
            t2i = t2i ** 2 % p
            if t2i == 1:
                break

        b = c ** (1 << (m - i - 1)) % p
        m = i
        c = b ** 2 % p
        t = (t * c) % p
        r = (r * b) % p

    return r

# Parameters
p = 751  # prime modulus
k = 20   # chosen small integer
a = -1   # coefficient of x in the elliptic curve
b = 188  # constant in the elliptic curve

# Elliptic curve equation: y^2 = x^3 + ax + b mod p
def koblitz_encode(m, max_attempts=1000):
    num = 1  # Start with x = m * k + 1
    attempts = 0

    while attempts < max_attempts:
        x = m * k + num
        rhs = (x**3 + a * x + b) % p  # right-hand side of the elliptic curve equation

        # Check if rhs is a quadratic residue modulo p
        if is_quadratic_residue(rhs, p):
            y = mod_sqrt(rhs, p)
            if y is not None:
                return (x, y)  # Return the point (x, y) as a tuple

        num += 1  # Increment to check next x value
        attempts += 1

    # If no valid point is found after max_attempts
    raise ValueError(f"No valid point found after {max_attempts} attempts for message {m}.")

# Decoding: m = (x - num) / k
def koblitz_decode(x, num=1):
    return (x - num) // k

# Example: encoding message m
message = 11
encoded_point = koblitz_encode(message)

# Decoding message
decoded_message = koblitz_decode(encoded_point[0])

# Output the result
print(f"Encoded point: {encoded_point}")
print(f"Decoded message: {decoded_message}")


Encoded point: (224, 503)
Decoded message: 11
